## Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...

С помощью pandas построим DataFrame и убедимся, что все 396632 строк с координатами считаны успешно.

In [1]:
import pandas as pd

In [2]:
lines = []
data = []

with open('data/checkins.dat') as input_file:
    columns = list(map(lambda x: x.strip(), input_file.readline().strip().split('|')))
    n_params = len(columns)
    for line in input_file:
        items = list(map(lambda x: x.strip(), line.strip().split('|')))
        if len(items) == n_params:
            data.append(items)

In [3]:
df = pd.DataFrame(data)
df.columns = columns
df = df[(df.latitude != '') & (df.longitude != '')]
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
7,984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22
9,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870,380645,33.4483771,-112.0740373,2012-04-21 17:38:18


In [4]:
df.to_csv('data/data.csv')

In [5]:
df.shape

(396634, 6)

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [6]:
from sklearn.cluster import MeanShift

In [7]:
data = df[['latitude', 'longitude']]
data = data.applymap(float)
data_sample = data.sample(100000)

In [8]:
%%time
clst = MeanShift(bandwidth = 0.1)
clst.fit(data_sample)

CPU times: user 3min 7s, sys: 2.58 s, total: 3min 9s
Wall time: 3min 16s


MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...

In [9]:
data_sample['cluster'] = clst.predict(data_sample)

cluster_size = pd.DataFrame(data_sample.pivot_table(index = 'cluster', aggfunc = 'count', values = 'latitude'))
cluster_size.columns = ['clust_size']

cluster_centers_df = pd.DataFrame(clst.cluster_centers_)
cluster_centers_df.columns = ['cent_latitude', 'cent_longitude']

cluster_df = cluster_centers_df.join(cluster_size)
cluster_df.to_csv('clusters.csv', index = None)
cluster_df = cluster_df[cluster_df.clust_size > 15]
cluster_df.head()

,cent_latitude,cent_longitude,clust_size
0,40.717715,-73.989307,14111
1,41.878173,-87.629840,3771
2,38.886945,-77.047944,2722
3,37.687440,-122.409229,3892
4,33.446843,-111.902718,2276


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [10]:
def get_distance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5

office_coordinates = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
]

def get_min_distance_to_office(lat, lon):
    min_dist = None
    for (of_lat, of_lon) in office_coordinates:
        dist = get_distance(lat, lon, of_lat, of_lon)
        if (min_dist is None) or (dist < min_dist):
            min_dist = dist
    return min_dist

In [11]:
cluster_df['min_distance'] = list(map(get_min_distance_to_office,\
                                      cluster_df.cent_latitude, cluster_df.cent_longitude))
cluster_df.sort_values(by='min_distance')[:20]

,cent_latitude,cent_longitude,clust_size,min_distance
262,-33.869841,151.210649,51,0.005686
269,52.371535,4.892007,48,0.010103
309,25.848346,-80.310833,43,0.023503
52,51.502754,-0.127210,298,0.051731
50,33.811670,-118.144968,317,0.074588
25,25.787122,-80.215945,680,0.134905
93,26.030162,-80.219998,140,0.192932
92,33.901317,-118.062797,136,0.195891
375,26.138358,-80.336668,34,0.270913
32,33.937668,-118.409857,406,0.289196
